#  Занятие 1 (расширенное): Пространство состояний и фильтры Калмана

Расширенный ноутбук рассчитан на 3–4 часа работы.

##  Цели занятия
- Понять, как временной ряд описывается в пространстве состояний.
- Разобрать полный цикл работы фильтра Калмана: предсказание, коррекция, сглаживание.
- Научиться реализовывать фильтр Калмана для одномерных и многомерных систем.
- Познакомиться с идеями расширенного фильтра Калмана (EKF) для нелинейных моделей.
- Применить фильтр Калмана к синтетическим и реальным данным.


## 0. Подготовка окружения


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('whitegrid')

print(' Библиотеки загружены')


## 1. Теория: модели пространства состояний

Модель пространства состояний описывается двумя уравнениями:

- **Уравнение состояния:**  
  \( x_t = A x_{t-1} + B u_t + w_t \), где \( w_t \sim \mathcal{N}(0, Q) \).
- **Уравнение наблюдений:**  
  \( y_t = H x_t + D u_t + v_t \), где \( v_t \sim \mathcal{N}(0, R) \).

Здесь \( x_t \) — скрытое состояние, \( y_t \) — наблюдение, \( A, H \) — матрицы динамики и наблюдения, \( Q, R \) — ковариации шумов.

Типичные примеры:
- Local level (только уровень).
- Local linear trend (уровень + тренд).
- Модели с сезонностью (дополнительные гармоники в состоянии).


## 2. Теория: алгоритм фильтра Калмана

Пусть на шаге \( t-1 \) известны \( \hat{x}_{t-1|t-1} \) и \( P_{t-1|t-1} \).

### 2.1 Шаг предсказания
- \( \hat{x}_{t|t-1} = A \hat{x}_{t-1|t-1} \)
- \( P_{t|t-1} = A P_{t-1|t-1} A^T + Q \)

### 2.2 Шаг коррекции
- Остаток: \( r_t = y_t - H \hat{x}_{t|t-1} \)
- Ковариация остатка: \( S_t = H P_{t|t-1} H^T + R \)
- Усиление Калмана: \( K_t = P_{t|t-1} H^T S_t^{-1} \)
- Обновление состояния: \( \hat{x}_{t|t} = \hat{x}_{t|t-1} + K_t r_t \)
- Обновление ковариации: \( P_{t|t} = (I - K_t H) P_{t|t-1} \)


## 3. Генерация синтетических данных (тренд + две сезонности + выбросы)


In [ ]:
np.random.seed(42)
n = 400
t = np.arange(n)

trend = 0.02 * t
season_short = 3 * np.sin(2 * np.pi * t / 24)
season_long = 1.5 * np.sin(2 * np.pi * t / 120)
true_state = trend + season_short + season_long

meas_noise = np.random.normal(0, 1.0, n)
observations = true_state + meas_noise

outlier_idx = [80, 81, 200, 280]
observations[outlier_idx] += np.array([10, -8, 12, -10])

plt.figure(figsize=(14, 5))
plt.plot(true_state, 'g-', lw=2, label='Истинное состояние')
plt.plot(observations, 'b.', ms=4, label='Наблюдения с шумом и выбросами')
plt.legend()
plt.title('Синтетический временной ряд для фильтра Калмана')
plt.show()


## 4. Скалярный фильтр Калмана (1D пример)


In [ ]:
def kalman_scalar(y, A=1.0, H=1.0, Q=0.05, R=1.0, x0=0.0, P0=1.0):
    y = np.asarray(y)
    n = len(y)
    x_est = np.zeros(n)
    P_est = np.zeros(n)
    x = x0
    P = P0
    for i in range(n):
        # Prediction
        x_pred = A * x
        P_pred = A * P * A + Q
        # Update
        y_pred = H * x_pred
        S = H * P_pred * H + R
        K = P_pred * H / S
        x = x_pred + K * (y[i] - y_pred)
        P = (1 - K * H) * P_pred
        x_est[i] = x
        P_est[i] = P
    return x_est, P_est

x_scalar, P_scalar = kalman_scalar(observations, Q=0.1, R=1.0)

plt.figure(figsize=(14, 5))
plt.plot(true_state, 'g-', lw=2, label='Истинное состояние')
plt.plot(observations, 'b.', ms=3, alpha=0.5, label='Наблюдения')
plt.plot(x_scalar, 'r-', lw=2, label='Скалярный Kalman')
plt.legend()
plt.title('Скалярный фильтр Калмана')
plt.show()


## 5. Матричный фильтр Калмана (уровень + тренд)


In [ ]:
class KalmanFilterMatrix:
    def __init__(self, A, H, Q, R, x0, P0):
        self.A = A
        self.H = H
        self.Q = Q
        self.R = R
        self.x = x0.copy()
        self.P = P0.copy()
        self.filtered_states = []
        self.filtered_covs = []
        self.predicted_states = []
        self.predicted_covs = []

    def step(self, y):
        # Prediction
        x_pred = self.A @ self.x
        P_pred = self.A @ self.P @ self.A.T + self.Q
        # Innovation
        y = np.asarray(y).reshape(-1, 1)
        y_pred = self.H @ x_pred
        v = y - y_pred
        S = self.H @ P_pred @ self.H.T + self.R
        K = P_pred @ self.H.T @ np.linalg.inv(S)
        # Update
        self.x = x_pred + K @ v
        I = np.eye(self.A.shape[0])
        self.P = (I - K @ self.H) @ P_pred
        # Save
        self.predicted_states.append(x_pred.copy())
        self.predicted_covs.append(P_pred.copy())
        self.filtered_states.append(self.x.copy())
        self.filtered_covs.append(self.P.copy())

    def filter(self, y_series):
        self.filtered_states = []
        self.filtered_covs = []
        self.predicted_states = []
        self.predicted_covs = []
        for y in y_series:
            self.step(y)
        return (np.array(self.filtered_states),
                np.array(self.filtered_covs),
                np.array(self.predicted_states),
                np.array(self.predicted_covs))

# Модель local linear trend
A = np.array([[1.0, 1.0],
              [0.0, 1.0]])
H = np.array([[1.0, 0.0]])
Q = np.diag([0.05, 0.01])
R = np.array([[1.0]])
x0 = np.array([[0.0], [0.0]])
P0 = np.eye(2) * 10

kf = KalmanFilterMatrix(A, H, Q, R, x0, P0)
f_states, f_covs, p_states, p_covs = kf.filter(observations)

level_est = f_states[:, 0, 0]
slope_est = f_states[:, 1, 0]

plt.figure(figsize=(14, 6))
plt.plot(true_state, 'g-', lw=2, label='Истинное состояние')
plt.plot(observations, 'b.', ms=3, alpha=0.5, label='Наблюдения')
plt.plot(level_est, 'r-', lw=2, label='Оценка уровня (Kalman)')
plt.legend()
plt.title('Матричный фильтр Калмана: уровень + тренд')
plt.show()


## 6. Сглаживание (RTS smoother)


In [ ]:
def rts_smoother(A, filtered_states, filtered_covs, predicted_states, predicted_covs):
    T = filtered_states.shape[0]
    x_smooth = np.zeros_like(filtered_states)
    P_smooth = np.zeros_like(filtered_covs)

    x_smooth[-1] = filtered_states[-1]
    P_smooth[-1] = filtered_covs[-1]

    for t in range(T - 2, -1, -1):
        P_f = filtered_covs[t]
        P_p = predicted_covs[t + 1]
        J = P_f @ A.T @ np.linalg.inv(P_p)
        x_smooth[t] = filtered_states[t] + J @ (x_smooth[t + 1] - predicted_states[t + 1])
        P_smooth[t] = P_f + J @ (P_smooth[t + 1] - P_p) @ J.T
    return x_smooth, P_smooth

x_smooth, P_smooth = rts_smoother(A, f_states, f_covs, p_states, p_covs)
level_smooth = x_smooth[:, 0, 0]

plt.figure(figsize=(14, 5))
plt.plot(true_state, 'g-', lw=2, label='Истинное состояние')
plt.plot(level_est, 'r--', lw=1.5, label='Фильтрация (онлайн)')
plt.plot(level_smooth, 'k-', lw=2, label='Сглаживание (offline)')
plt.legend()
plt.title('Сравнение фильтрации и сглаживания Kalman')
plt.show()


## 7. Многомерный пример: движение точки в 2D


In [ ]:
np.random.seed(7)
T = 200
dt = 1.0

x_true = np.zeros((T, 4, 1))  # [x, vx, y, vy]
x_true[0] = np.array([[0.0], [1.0], [0.0], [0.5]])

A2 = np.array([[1, dt, 0,  0],
               [0,  1, 0,  0],
               [0,  0, 1, dt],
               [0,  0, 0,  1]])
Q2 = np.eye(4) * 0.01
H2 = np.array([[1, 0, 0, 0],
               [0, 0, 1, 0]])
R2 = np.eye(2) * 0.5

for i in range(1, T):
    process_noise = np.random.multivariate_normal(np.zeros(4), Q2).reshape(4, 1)
    x_true[i] = A2 @ x_true[i - 1] + process_noise

y_meas = []
for i in range(T):
    meas_noise = np.random.multivariate_normal(np.zeros(2), R2).reshape(2, 1)
    y_meas.append(H2 @ x_true[i] + meas_noise)
y_meas = np.array(y_meas)

x0_2 = np.array([[0.0], [0.0], [0.0], [0.0]])
P0_2 = np.eye(4) * 5
kf2 = KalmanFilterMatrix(A2, H2, Q2, R2, x0_2, P0_2)

kf2.filtered_states = []
kf2.filtered_covs = []
kf2.predicted_states = []
kf2.predicted_covs = []
for i in range(T):
    kf2.step(y_meas[i])
f_s2 = np.array(kf2.filtered_states)

plt.figure(figsize=(7, 7))
plt.plot(x_true[:, 0, 0], x_true[:, 2, 0], 'g-', lw=2, label='Истинная траектория')
plt.scatter(y_meas[:, 0, 0], y_meas[:, 1, 0], c='b', s=10, alpha=0.4, label='Наблюдения')
plt.plot(f_s2[:, 0, 0], f_s2[:, 2, 0], 'r-', lw=2, label='Kalman траектория')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Двумерное движение и фильтр Калмана')
plt.axis('equal')
plt.show()


## 8. Нелинейный пример и идея EKF


In [ ]:
def f_nonlinear(x):
    return x + 0.1 * np.sin(x)

def h_nonlinear(x):
    return x**2

def ekf(y, Q=0.05, R=0.5, x0=0.0, P0=1.0):
    y = np.asarray(y)
    n = len(y)
    x_est = np.zeros(n)
    P_est = np.zeros(n)
    x = x0
    P = P0
    for i in range(n):
        # Prediction
        x_pred = f_nonlinear(x)
        F = 1 + 0.1 * np.cos(x)  # Якоби по f
        P_pred = F * P * F + Q
        # Update
        H = 2 * x_pred  # Якоби по h
        y_pred = h_nonlinear(x_pred)
        S = H * P_pred * H + R
        K = P_pred * H / S
        x = x_pred + K * (y[i] - y_pred)
        P = (1 - K * H) * P_pred
        x_est[i] = x
        P_est[i] = P
    return x_est, P_est

np.random.seed(10)
n_nl = 150
x_true_nl = np.zeros(n_nl)
x_true_nl[0] = 0.5
for i in range(1, n_nl):
    x_true_nl[i] = f_nonlinear(x_true_nl[i-1]) + np.random.normal(0, 0.1)
y_nl = x_true_nl**2 + np.random.normal(0, 0.5, n_nl)

x_ekf, P_ekf = ekf(y_nl, Q=0.05, R=0.5)

plt.figure(figsize=(14, 5))
plt.plot(x_true_nl, 'g-', lw=2, label='Истинное состояние (нелинейное)')
plt.plot(y_nl, 'b.', alpha=0.5, label='Наблюдения (h(x)=x^2)')
plt.plot(x_ekf, 'r-', lw=2, label='EKF оценка состояния')
plt.legend()
plt.title('Пример работы расширенного фильтра Калмана (EKF)')
plt.show()


## 9. Итоговые задания (на 3–4 часа)

### 🔹 Задание 1 (теория, ≈30 мин)
- Выпишите матрицы A, H, Q, R для:
  1. Local level.
  2. Local linear trend.
  3. Local linear trend + сезонность периода 12.
- Объясните, как изменение Q и R влияет на гладкость и скорость реакции фильтра.

### 🔹 Задание 2 (эксперименты с параметрами, ≈40 мин)
- Для синтетического ряда из раздела 3 проведите сеточный поиск по Q ∈ {0.01, 0.1, 0.5} и R ∈ {0.5, 1, 3}.
- Для каждой пары параметров посчитайте MAE и RMSE между истинным состоянием и оценкой фильтра.
- Постройте heatmap ошибок и сделайте вывод, какие параметры дают лучший баланс сглаживания и реакции.

### 🔹 Задание 3 (реальные данные, ≈45–60 мин)
- Загрузите реальный временной ряд (цены, трафик, нагрузки и т.п.).
- Реализуйте модель local linear trend с фильтром Калмана для этих данных.
- Постройте графики: наблюдаемый ряд, оценка уровня, оценка тренда.
- Добавьте сглаживание и сравните результаты с фильтрацией.

### 🔹 Задание 4 (многомерный кейс, ≈40–50 мин)
- Модифицируйте двумерный пример движения:
  1. Добавьте ускорение в состояние ([x, vx, ax, y, vy, ay]).
  2. Сгенерируйте траекторию с переменной скоростью.
  3. Реализуйте фильтр Калмана и сравните точность координат с исходной моделью.

### 🔹 Задание 5 (EKF‑мини‑проект, ≈45–60 мин)
- Измените функции f(x) и h(x) в примере EKF на свои (например, добавьте нелинейную сезонность).
- Аккуратно выведите Якобианы и реализуйте их в коде.
- Подберите параметры Q и R так, чтобы фильтр был устойчивым и давал разумные оценки.
- Сравните EKF с простой фильтрацией скользящим средним.

### 🔹 Бонус (по желанию)
- Реализуйте онлайн‑версию фильтра Калмана в виде функции, которая принимает одно новое наблюдение и возвращает обновлённое состояние, имитируя потоковые данные.
